In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd

In [2]:
dataTrain=pd.read_csv("allAtt_onehot_large_train.csv")
dataTest=pd.read_csv("allAtt_onehot_large_test.csv")
print(dataTrain.head())
print(dataTrain.shape)

   FTR  HTGS  ATGS  HTGC  ATGC  HTP  ATP  HM1  HM2  HM3  ...  ATWinStreak5  \
0    0   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2  ...             0   
1    0   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2  ...             0   
2    1   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2  ...             0   
3    0   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2  ...             0   
4    1   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2  ...             0   

   ATLossStreak3  ATLossStreak5  HTGD  ATGD  DiffPts  DiffFormPts  DiffLP  \
0              0              0   0.0   0.0      0.0          0.0   -11.0   
1              0              0   0.0   0.0      0.0          0.0     2.0   
2              0              0   0.0   0.0      0.0          0.0     2.0   
3              0              0   0.0   0.0      0.0          0.0   -17.0   
4              0              0   0.0   0.0      0.0          0.0     4.0   

   final1  final2  
0     1.0     0.0  
1     1.0     0.0  
2     0.

In [8]:
batch_size = 64
# Each MNIST image batch is a tensor of shape (batch_size, 28, 1).
# Each input sequence will be of size (28, 1).
input_dim = 27

units = 64
output_size = 2  # labels are from Win or Loss

# Build the RNN model
def build_model(allow_cudnn_kernel=True):
    # CuDNN is only available at the layer level, and not at the cell level.
    # This means `LSTM(units)` will use the CuDNN kernel,
    # while RNN(LSTMCell(units)) will run on non-CuDNN kernel.
    if allow_cudnn_kernel:
        # The LSTM layer with default options uses CuDNN.
        lstm_layer = keras.layers.LSTM(units, input_shape=(input_dim,1))
    else:
        # Wrapping a LSTMCell in a RNN layer will not use CuDNN.
        lstm_layer = keras.layers.RNN(
            keras.layers.LSTMCell(units), input_shape=(input_dim,1)
        )
    model = keras.models.Sequential(
        [
            lstm_layer,
            keras.layers.BatchNormalization(),
            keras.layers.Dense(output_size),
        ]
    )
    return model

In [9]:
x_train, y_train = dataTrain.iloc[:,1:28].values,dataTrain.iloc[:,28:].values
x_train=np.reshape(x_train,(1860,27,1))
x_test, y_test = dataTest.iloc[:,1:28].values,dataTest.iloc[:,28:].values
x_test=np.reshape(x_test,(800,27,1))

In [10]:
model = build_model(allow_cudnn_kernel=True)

model.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer="Adam",
    metrics=["categorical_accuracy"],
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 64)                16896     
                                                                 
 batch_normalization_1 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 17,282
Trainable params: 17,154
Non-trainable params: 128
_________________________________________________________________


In [11]:
model.fit(
    x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, epochs=10
)


Epoch 1/10
30/30 [==============================] - 4s 95ms/step - loss: 0.6652 - categorical_accuracy: 0.6672 - val_loss: 0.6646 - val_categorical_accuracy: 0.6463
Epoch 2/10
30/30 [==============================] - 0s 12ms/step - loss: 0.6087 - categorical_accuracy: 0.6710 - val_loss: 0.6540 - val_categorical_accuracy: 0.6575
Epoch 3/10
30/30 [==============================] - 0s 12ms/step - loss: 0.6058 - categorical_accuracy: 0.6758 - val_loss: 0.6508 - val_categorical_accuracy: 0.6625
Epoch 4/10
30/30 [==============================] - 0s 12ms/step - loss: 0.6038 - categorical_accuracy: 0.6758 - val_loss: 0.6472 - val_categorical_accuracy: 0.6600
Epoch 5/10
30/30 [==============================] - 0s 12ms/step - loss: 0.6054 - categorical_accuracy: 0.6694 - val_loss: 0.6399 - val_categorical_accuracy: 0.6650
Epoch 6/10
30/30 [==============================] - 0s 13ms/step - loss: 0.6039 - categorical_accuracy: 0.6688 - val_loss: 0.6352 - val_categorical_accuracy: 0.6612
Epoch 7/10